## 타이타닉 데이터셋 도전

- 승객의 나이, 성별, 승객 등급, 승선 위치 같은 속성을 기반으로 하여 승객의 생존 여부를 예측하는 것이 목표

- [캐글](https://www.kaggle.com)의 [타이타닉 챌린지](https://www.kaggle.com/c/titanic)에서 `train.csv`와 `test.csv`를 다운로드
- 두 파일을 각각 datasets 디렉토리에 titanic_train.csv titanic_test.csv로 저장

### 1. 데이터 적재

In [1]:
import pandas as pd
train_data = pd.read_csv("datasets/titanic_train.csv")
test_data = pd.read_csv("datasets/titanic_test.csv")

### 2. 데이터 탐색

#### train_data 살펴보기

In [2]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


* **Survived**: 타깃. 0은 생존하지 못한 것이고 1은 생존을 의미
* **Pclass**: 승객 등급. 1, 2, 3등석.
* **Name**, **Sex**, **Age**: 이름 그대로의 의미
* **SibSp**: 함께 탑승한 형제, 배우자의 수
* **Parch**: 함께 탑승한 자녀, 부모의 수
* **Ticket**: 티켓 아이디
* **Fare**: 티켓 요금 (파운드)
* **Cabin**: 객실 번호
* **Embarked**: 승객이 탑승한 곳. C(Cherbourg), Q(Queenstown), S(Southampton)


#### 누락 데이터 살펴보기

In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


- **Age**, **Cabin**, **Embarked** 속성의 일부가 null
- 특히 **Cabin**은 77%가 null. 일단 **Cabin**은 무시하고 나머지를 활용
- **Age**는 177개(19%)가 null이므로 이를 어떻게 처리할지 결정해야 함 - null을 중간 나이로 바꾸기 고려
- **Name**과 **Ticket** 속성은 숫자로 변환하는 것이 조금 까다로와서 지금은 무시

#### 통계치 살펴보기

In [4]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


* 38%만 **Survived**
* 평균 **Fare**는 32.20 파운드
* 평균 **Age**는 30보다 적음

#### Survived(머신러닝에서 타깃)가 0과 1로 이루어졌는지 확인

In [5]:
train_data.Survived.value_counts()

0    549
1    342
Name: Survived, dtype: int64

#### 범주형(카테고리) 특성들을 확인

In [6]:
train_data.Pclass.value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [7]:
train_data.Sex.value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [8]:
train_data.Embarked.value_counts() #사우스햄튼에서 많이 왔다.

S    644
C    168
Q     77
Name: Embarked, dtype: int64

**Embarked** 특성은 승객이 탑승한 곳 : C=Cherbourg, Q=Queenstown, S=Southampton.

### 3. 데이터 탐색 (상세)

#### Name과 Age 열 을 Age 순으로 정렬해서 보기

In [9]:
train_data[['Name', 'Age']].sort_values(by=['Age'], axis=0)

,Name,Age
803,"Thomas, Master. Assad Alexander",0.42
755,"Hamalainen, Master. Viljo",0.67
644,"Baclini, Miss. Eugenie",0.75
469,"Baclini, Miss. Helene Barbara",0.75
78,"Caldwell, Master. Alden Gates",0.83
...,...,...
859,"Razi, Mr. Raihed",NaN
863,"Sage, Miss. Dorothy Edith ""Dolly""",NaN
868,"van Melkebeke, Mr. Philemon",NaN
878,"Laleff, Mr. Kristo",NaN


#### 나이(Age)가 60 이상인 사람들의 Name과 Age 확인해 보기

In [10]:
train_data[['Name','Age']][train_data['Age'] >= 60]

,Name,Age
33,"Wheadon, Mr. Edward H",66.0
54,"Ostby, Mr. Engelhart Cornelius",65.0
96,"Goldschmidt, Mr. George B",71.0
116,"Connors, Mr. Patrick",70.5
170,"Van der hoef, Mr. Wyckoff",61.0
252,"Stead, Mr. William Thomas",62.0
275,"Andrews, Miss. Kornelia Theodosia",63.0
280,"Duane, Mr. Frank",65.0
326,"Nysveen, Mr. Johan Hansen",61.0
366,"Warren, Mrs. Frank Manley (Anna Sophia Atkinson)",60.0


#### 나이가(Age)가 60 보다 많고 1등석에 탔으며 여성인 탑승자 확인해 보기

In [11]:
train_data[(train_data['age'] >= 60) and (train_data['Pclass'] == 1) and (train_data['female'] == 'female')]

KeyError: 'age'

#### 요금(Fare)의 최대값 최소값 확인해 보기

In [14]:
train_data.Fare.max()

512.3292

In [15]:
train_data.Fare.min()

0.0

#### 등급(Pcalss) 그룹별 생존률 확인해보기

In [16]:
train_data.groupby('Pclass').size()

Pclass
1    216
2    184
3    491
dtype: int64

In [17]:
train_data.groupby('Pclass').size()

Pclass
1    216
2    184
3    491
dtype: int64

In [18]:
train_data.groupby(['Pclass', 'survived']).mean()

KeyError: 'survived'

In [19]:
train_data.groupby('Pclass')['survived'].mean()

KeyError: 'Column not found: survived'

In [ ]:
train_data

#### 특성을 조합해 또다른 특성(RelativesOnboard)을 만들기(가족과 탑승한 사람과 혼자 탑승한 사람)

In [45]:
train_data['RelativesOnboard'] = train_data.SibSp + train_data.Parch + 1

In [46]:
train_data['RelativesOnboard'].value_counts()

1     537
2     161
3     102
4      29
6      22
5      15
7      12
11      7
8       6
Name: RelativesOnboard, dtype: int64

#### RelativesOnboard 그룹의 생존률 평군 알아보기

In [ ]:
train_data.groupby('RelativesOnboard').Survived.mean()

### 4. 데이터 전처리 (누락 데이터 처리, 범주화 등)

#### Cabin 열 : 전체 삭제하기

In [21]:
train_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

### 열 삭제하는 방법

In [22]:
train_data.dropna(thresh= 500, axis=1, inplace=True #널값이 500개 이상인 열을 삭제

SyntaxError: unexpected EOF while parsing (<ipython-input-22-d1097471ad29>, line 1)

In [47]:
train_data.drop('Carbin', axis=1, inplace=True)

KeyError: "['Carbin'] not found in axis"

In [48]:
train_data.pop('Cabin')

0       NaN
1       C85
2       NaN
3      C123
4       NaN
       ... 
886     NaN
887     B42
888     NaN
889    C148
890     NaN
Name: Cabin, Length: 891, dtype: object

#### Embarked 열 : 승선도시 최고 빈도수 값으로 대체하기

In [ ]:
train_data.Embarked.value_counts(dropna=False)

In [24]:
# embark_town 열의 NaN값을 승선도시 중에서 가장 많이 출현한 값으로 치환하기
most_freq = #########
most_freq

SyntaxError: invalid syntax (<ipython-input-24-badcad41d9f3>, line 2)

In [49]:
a = train_data['Embarked'].mode()[0]
a = train_data['Embarked'].value_counts().idxmax()

In [50]:
train_data['Embarked'].fillna(a, inplace=True)

In [51]:
train_data.Embarked.value_counts(dropna=False)

S    646
C    168
Q     77
Name: Embarked, dtype: int64

#### Age 열 : 중간값으로 대체하기

In [52]:
train_data.Age.describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [53]:
a = train_data['Age'].median()
a

28.0

In [54]:
train_data['Age'].fillna(a, inplace=True)

In [55]:
train_data['Age'].isnull().sum()

0

#### Age 열: 범주로 나눠보기

In [56]:
import numpy as np
bins = [0,18, 25, 35, 60, 80]
group_names = ['Children','Youth', 'YoungAdult', 'MiddleAged', 'Senior']
age_cats= pd.cut(train_data.Age, bins=bins, labels=group_names)
age_cats

0           Youth
1      MiddleAged
2      YoungAdult
3      YoungAdult
4      YoungAdult
          ...    
886    YoungAdult
887         Youth
888    YoungAdult
889    YoungAdult
890    YoungAdult
Name: Age, Length: 891, dtype: category
Categories (5, object): ['Children' < 'Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

In [57]:
age_cats.value_counts()

YoungAdult    373
MiddleAged    195
Youth         162
Children      139
Senior         22
Name: Age, dtype: int64

#### Age 열: 더미변수로 바꾸기

In [33]:
Age_dummies = pd.get_dummies(age_cats)
Age_dummies= Age_dummies.add_prefix('Age_')
Age_dummies

,Age_Children,Age_Youth,Age_YoungAdult,Age_MiddleAged,Age_Senior
0,0,1,0,0,0
1,0,0,0,1,0
2,0,0,1,0,0
3,0,0,1,0,0
4,0,0,1,0,0
...,...,...,...,...,...
886,0,0,1,0,0
887,0,1,0,0,0
888,0,0,1,0,0
889,0,0,1,0,0


#### 중복 데이터 확인

In [34]:
train_data.duplicated().sum()

0

#### ["Pclass", "Sex", "Embarked"] 에 대해 각각 One-hot Encoding

In [35]:
Pclass_dummies = pd.get_dummies(pd.cut(train_data.Pclass, bins=3, labels=['1', '2','3']))
Pclass_dummies = Pclass_dummies.add_prefix('Pclass_')
Pclass_dummies

,Pclass_1,Pclass_2,Pclass_3
0,0,0,1
1,1,0,0
2,0,0,1
3,1,0,0
4,0,0,1
...,...,...,...
886,0,1,0
887,1,0,0
888,0,0,1
889,1,0,0


In [36]:
Sex_dummies = pd.get_dummies(train_data.Sex)
Sex_dummies = Sex_dummies.add_prefix('Sex_')
Sex_dummies

,Sex_female,Sex_male
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1
...,...,...
886,0,1
887,1,0
888,1,0
889,0,1


In [37]:
Embarked_dummies = pd.get_dummies(train_data.Embarked)
Embarked_dummies = Embarked_dummies.add_prefix('Embarked_')
Embarked_dummies

,Embarked_C,Embarked_Q,Embarked_S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1
...,...,...,...
886,0,0,1
887,0,0,1
888,0,0,1
889,1,0,0


#### 레이블 가져오기

In [38]:
y_train = train_data.Survived

#### 위에서 만들어진 더미 데이터들 프레임을 합치기

In [39]:
train_data = train_data.drop(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', "Ticket", "Embarked" ], axis=1)

In [40]:
X_train = pd.concat([train_data, Age_dummies, Pclass_dummies, Sex_dummies, Embarked_dummies], axis=1)

In [41]:
X_train.shape

(891, 17)

In [42]:
X_train.head()

,SibSp,Parch,Fare,Cabin,Age_Children,Age_Youth,Age_YoungAdult,Age_MiddleAged,Age_Senior,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,7.2500,NaN,0,1,0,0,0,0,0,1,0,1,0,0,1
1,1,0,71.2833,C85,0,0,0,1,0,1,0,0,1,0,1,0,0
2,0,0,7.9250,NaN,0,0,1,0,0,0,0,1,1,0,0,0,1
3,1,0,53.1000,C123,0,0,1,0,0,1,0,0,1,0,0,0,1
4,0,0,8.0500,NaN,0,0,1,0,0,0,0,1,0,1,0,0,1


#### 분류기 훈련

#### 교차 검증으로 평가

#### RandomForestClassifier 적용

#### 예측 결과를 CSV 파일로 만들어 업로드
(이 때 test_data도 train_data 전처리 과정을 거친후 예측)

In [58]:
import pandas as pd
train_data = pd.read_csv("datasets/titanic_train.csv")
test_data = pd.read_csv("datasets/titanic_test.csv")

### 전처리 파이프 라인

* 특성과 레이블 분리

In [59]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [60]:
train_data.info() #age랑 carbin걸러.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [64]:
train_num = train_data[['Age', 'SibSp','Parch','Fare']]
train_cat = train_data[['Pclass', 'Sex', 'Embarked']]

In [66]:
#수치형 특성 전처리 1
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
    ])

train_num_tr = num_pipeline.fit_transform(train_num)

In [75]:
train_num_tr.shape

(891, 4)

* 범주형 파이프 라인 구성

In [ ]:
# 1. 누락치를 most_frequent(가장 많은 값)으로 대체

In [ ]:
# 2. OneHotEncoding을 한다.

In [71]:
cat_pipeline = Pipeline([       
        ('imputer', SimpleImputer(strategy="most_frequent")),
        ('cat', OneHotEncoder(sparse=False)) #sparse가 True면 generator처럼 희소한 데이터에 대해 보여주지 않는다 허나 False면 굳이 안보여줘도 되지만 보여준다..!
        ])
train_cat_tr = cat_pipeline.fit_transform(train_cat)

In [74]:
train_cat_tr.shape

(891, 8)

* 수치형 파이프 라인 구성

In [ ]:
# 1. 누락치를 median(중간값)으로 대체

In [ ]:
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
    ])
train_num_tr = num_pipeline.

* 범주형 파이프라인 + 수치형 파이프라인 transform하면됌

In [76]:
num_attribs = ['Age', 'SibSp','Parch','Fare'] #수치형 파이프라인

cat_attribs =['Pclass', 'Sex', 'Embarked'] #범주형 파이프라인

In [83]:
a = np.c_[train_cat_tr, train_num_tr]

In [85]:
a.shape

(891, 12)

In [ ]:
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): # *args 또는 **kargs 없음
        return self # x
    def fit(self, X, y=None):
        return self  # x
    def transform(self, X, Y):
        return np.c_[]

In [81]:
full_pipeline = pd.concat([
    ('num', num_pipeline, num_attribs),
    ('cat', cat_pipeline, cat_attribs) #범주형이 원래부터 1개여서 따로 파이프라인 안만들고 결합해도 된다.
    ])

TypeError: cannot concatenate object of type '<class 'tuple'>'; only Series and DataFrame objs are valid